In [15]:
!pip install sentence-transformers tensorflow transformers

In [16]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.model_selection import train_test_split

tokenizer = GPT2Tokenizer.from_pretrained("ashiqabdulkhader/GPT2-Malayalam")

embedding_model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
# embedding_model = AutoModelForCausalLM.from_pretrained("ashiqabdulkhader/GPT2-Malayalam", from_tf=True)

In [17]:
df = pd.read_csv("data.csv", delimiter=",")
df = df[df['Text'].apply(lambda x: isinstance(x, str))]
df = df.drop(['Website','Date'],axis=1)
df = df.dropna()
print(df.shape)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
max_length = 500

(1960, 9)


In [18]:
train_encodings = tokenizer(train_df['Text'].tolist(), truncation=True, padding=True, max_length=max_length, return_tensors='tf')
# train_embeddings = embedding_model(**train_encodings).last_hidden_state[:, 0, :]
train_embeddings = embedding_model.encode(train_df['Text'].tolist(), show_progress_bar=True)

test_encodings = tokenizer(test_df['Text'].tolist(), truncation=True, padding=True, max_length=max_length, return_tensors='tf')
test_embeddings = embedding_model.encode(test_df['Text'].tolist(), show_progress_bar=True)
# test_embeddings = embedding_model(**test_encodings).last_hidden_state[:, 0, :]

train_labels = train_df[['Hate','Misleading', 'Disinformation', 'Rumour', 'Sensationalism']].to_numpy()
test_labels = test_df[['Hate','Misleading', 'Disinformation', 'Rumour', 'Sensationalism']].to_numpy()

Batches:   0%|          | 0/49 [00:00<?, ?it/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

model = Sequential([
    BatchNormalization(),
    Dense(512, activation='relu', input_dim=train_embeddings.shape[1]),
    BatchNormalization(),
    Dense(256, activation="relu"),
    BatchNormalization(),
    Dense(5, activation="sigmoid"),
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_embeddings, train_labels, epochs=30, batch_size=8, validation_split=0.1)
model.save("first_iter.keras")

test_loss, test_accuracy = model.evaluate(test_embeddings, test_labels)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

Epoch 1/30
177/177 [==============================] - 4s 7ms/step - loss: 0.6389 - accuracy: 0.2240 - val_loss: 0.4980 - val_accuracy: 0.1783
Epoch 2/30
177/177 [==============================] - 1s 6ms/step - loss: 0.4925 - accuracy: 0.2587 - val_loss: 0.5122 - val_accuracy: 0.1783
Epoch 3/30
177/177 [==============================] - 1s 6ms/step - loss: 0.4751 - accuracy: 0.2856 - val_loss: 0.5127 - val_accuracy: 0.2484
Epoch 4/30
177/177 [==============================] - 1s 6ms/step - loss: 0.4687 - accuracy: 0.2884 - val_loss: 0.5075 - val_accuracy: 0.2930
Epoch 5/30
177/177 [==============================] - 1s 6ms/step - loss: 0.4570 - accuracy: 0.2856 - val_loss: 0.5507 - val_accuracy: 0.2484
Epoch 6/30
177/177 [==============================] - 1s 6ms/step - loss: 0.4539 - accuracy: 0.2785 - val_loss: 0.5843 - val_accuracy: 0.1783
Epoch 7/30
177/177 [==============================] - 1s 6ms/step - loss: 0.4483 - accuracy: 0.2955 - val_loss: 0.5912 - val_accuracy: 0.2866
Epoch 

In [24]:
import numpy as np

input_text = "സ്ത്രീധനത്തിന്റെ പേരിലുള്ള ആത്മഹത്യകളും പീഡനങ്ങളും എന്നും വാര്‍ത്തകളാകുമ്പോഴാണ് ബീഹാറില്‍ നിന്ന് മറ്റൊരു വാര്‍ത്ത. പറഞ്ഞുറപ്പിച്ച മുഴുവന്‍ സ്ത്രീധനവും നല്‍കിയില്ലെങ്കില്‍ വിവാഹം നടക്കില്ലെന്ന് വരന്‍. ആവശ്യപ്പെട്ട സ്ത്രീധനത്തിന്റെ കുറച്ച് കൊണ്ടു വന്നിട്ടുണ്ടെന്നും ബാക്കി പിന്നീട് നല്‍കാമെന്നും വധുവിന്റെ വീട്ടുകാര്‍ പറഞ്ഞതോടെയാണ് പ്രശ്‌നങ്ങള്‍ക്ക് തുടക്കമായത്."

# input_encoding = tokenizer([input_text], truncation=True, padding=True, max_length=max_length, return_tensors='tf')
input_embedding = embedding_model.encode([input_text])

predictions = model.predict(input_embedding)

predicted_probabilities = predictions[0]

class_labels = ['Hate', 'Misleading', 'Disinformation', 'Rumour', 'Sensationalism']
for label, prob in zip(class_labels, predicted_probabilities):
    print(f'Probability for {label}: {prob:.4f}')


1/1 [==============================] - 0s 17ms/step
Probability for Hate: 0.9858
Probability for Misleading: 0.8656
Probability for Disinformation: 0.0464
Probability for Rumour: 0.0841
Probability for Sensationalism: 0.7517
